# Aplicaciones a Problemas Directos

**Por:** David Ortiz, Rodrigo Salas

**Edición:** David Ortiz, Tabita Catalán, Tomás Banduc

### Introducción
Continuando con nuestro trabajo previo con PINNs en modelos no lineales, esta actividad amplía nuestra exploración hacia ecuaciones diferenciales parciales (EDPs) utilizando PINNs. Nos centraremos en la solución de un modelo de difusión lineal, específicamente la ecuación de calor en 1D, para examinar cómo las PINNs pueden aplicarse a problemas lineales más simples y profundizar en nuestra comprensión de su adaptabilidad a diferentes tipos de modelos de EDP.

### Resumen de la Actividad

En esta actividad, implementaremos una PINN para resolver la ecuación de calor, un modelo de difusión lineal, en una dimensión de espacio. Esto nos permitirá explorar la capacidad de las PINNs para resolver EDPs lineales de manera efectiva.

### Objetivos de la Actividad

Al final de la actividad, será capaz de:

- Definir problemas de EDP y construir soluciones analíticas ajustadas al modelo.
- Entrenar una PINN para estimar EDPs lineales con un enfoque libre de datos.

### Descripción Matemática del Problema

Esta actividad se centra en el modelo de difusión unidimensional, comunmente conocido como modelo de calor, que se rige por la ecuación de calor

$$
\begin{alignat*}{3}
    \frac{\partial u}{\partial t} &&= \kappa\frac{\partial^2 u}{\partial x^2} + f(t,x), \quad && x \in [-1, 1], \quad t \in [0, 2], \quad \kappa \in \mathbb{R}
\end{alignat*}
$$

donde $u(t,x)$ representa una cantidad física de interés (temperatura, concentración, voltaje, etc.) en la posición $x \in [-1,1]$ en el tiempo $t \in [0,2]$, $\kappa$ es el coeficiente de difusión y $f(t,x)$ es un término fuente.

Para simplificar el problema, construiremos una solución analítica al modelo y supondremos, sin pérdida de generalidad, que $\kappa = 1$. Proponemos la siguiente función como solución del problema:

$$
u(t,x) = e^{-t}\sin(\pi x)
$$

Sustituyendo $u$ en la EDP, se deriva el siguiente problema con sus respectivas condiciones iniciales y de borde:

$$
\begin{alignat*}{3}
    \text{EDP:} \quad & \frac{\partial u}{\partial t} &&= \kappa\frac{\partial^2 u}{\partial x^2} - e^{-t}(\sin(\pi x) - \pi^2\sin(\pi x)), \quad && x \in [-1, 1], \quad t \in [0, 2], \quad \kappa \in \mathbb{R} \\
    \text{CI:} \quad & u(0,x) &&= \sin(\pi x), && x \in [-1, 1] \\
    \text{CB:} \quad & u(t,-1) &&= u(t,1) = 0, && t \in [0, 2] \\
    \text{Solución:} \quad & u(t,x) &&= e^{-t} \sin(\pi x)
\end{alignat*}
$$

## Flujo de Trabajo

1. Calcular la solución analítica utilizando un mallado uniforme.
2. Muestrear el dominio para entrenar la PINN.
3. Definir la función de pérdida informada con física y entrenar la PINN.


### Configuración Inicial

Comenzamos importando módulos y definiendo algunas funciones de utilidad

In [ ]:
%matplotlib inline
#%matplotlib widget

In [ ]:
# NumPy para operaciones numéricas
import numpy as np
# PyTorch para construir y entrenar redes neuronales
import torch
import torch.nn as nn
import torch.optim as optim
# Matplotlib para graficar
import matplotlib.pyplot as plt
import matplotlib as mlp
# Time para medir tiempo de entrenamiento
import time
# Warnings para ignorar mensajes de advertencia
import warnings
warnings.filterwarnings("ignore")

from matplotlib import animation, rc
from IPython.display import HTML
from scipy.stats import qmc

In [ ]:
warnings.filterwarnings("ignore")

# Actualización de los parámetros de Matplotlib
gray = '#5c5c5c' #'#5c5c5c' '000'
mlp.rcParams.update(
    {
        "image.cmap" : 'viridis', # plasma, inferno, magma, cividis
        "text.color" : gray,
        "xtick.color" :gray,
        "ytick.color" :gray,
        "axes.labelcolor" : gray,
        "axes.edgecolor" :gray,
        "axes.spines.right" : False,
        "axes.spines.top" : False,
        "axes.formatter.use_mathtext": True,
        "axes.unicode_minus": False,

        'font.size' : 16,
        'interactive': False,
        "font.family": 'sans-serif',
        "legend.loc" : 'best',
        'text.usetex': False,
        'mathtext.fontset': 'stix',
    }
)

# Definir pi en torch
torch.pi = torch.acos(torch.zeros(1)).item() * 2

# Error l2 relativo
def relative_l2_error(u_num, u_ref):
    # Calcular norma l2 de diferencia
    l2_diff = torch.norm(u_num - u_ref, p=2)

    # Calcular norma l2 de referencia
    l2_ref = torch.norm(u_ref, p=2)

    # Calcular norma l2 relativa
    relative_l2 = l2_diff / l2_ref
    return relative_l2

# Función para graficar soluciones
def plot_comparison(u_true, u_pred, loss):

    # Convertir tensores de numpy a arreglos, para graficar
    u_pred_np = u_pred.detach().numpy()

    # Crear figura con 2 subplots
    fig1, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Graficar solución analítica
    im1 = axs[0].imshow(u_true, extent=[-1,1,1,0])
    axs[0].set_title('Analytic solution for diffusion')
    axs[0].set_xlabel(r'$x$')
    axs[0].set_ylabel(r'$t$')
    fig1.colorbar(im1, spacing='proportional',
                            shrink=0.5, ax=axs[0])

    # Graficar predicción
    im2 = axs[1].imshow(u_pred_np, extent=[-1,1,1,0])
    axs[1].set_title('PINN solution for diffusion')
    axs[1].set_xlabel(r'$x$')
    axs[1].set_ylabel(r'$t$')
    fig1.colorbar(im2, spacing='proportional',
                            shrink=0.5, ax=axs[1])
    # Display de gráfico
    plt.tight_layout()
    plt.show()


    # Graficar los valores de la pérdida guardados durante el entrenamiento
    # Crear figura con 2 subplots
    fig2, axs = plt.subplots(1, 2, figsize=(12, 6))
    # Graficar diferencia entre valores reales y predicción
    difference = np.abs(u_true - u_pred_np)
    im3 = axs[0].imshow(difference, extent=[-1,1,1,0])
    axs[0].set_title(r'$|u(t,x) - u_{pred}(t,x)|$')
    axs[0].set_xlabel(r'$x$')
    axs[0].set_ylabel(r'$t$')
    fig2.colorbar(im3, spacing='proportional',
                            shrink=0.5, ax=axs[0])

    axs[1].plot(loss)
    axs[1].set_xlabel('Iteration')
    axs[1].set_ylabel('Loss')
    axs[1].set_yscale('log')
    axs[1].set_xscale('log')
    axs[1].set_title('Training Progress')
    axs[1].grid(True)

    # Display del gráfico
    plt.tight_layout()
    plt.show()

# Función para calcular gradientes con diferenciación automática
def grad(outputs, inputs):
    """Calcula derivadas parciales de un output con respecto a un input.
    Args:
        outputs: tensor (N, 1)
        inputs: tensor (N, D)
    """
    return torch.autograd.grad(outputs, inputs,
                        grad_outputs=torch.ones_like(outputs),
                        create_graph=True,
                        retain_graph=True,
                        )[0]

## 1. Solución Analítica
Utilizamos la solución $u(t,x) = e^{-t}\sin(\pi x)$ para el problema de difusión y la evaluamos en coordenadas específicas.

In [ ]:
def animate(x,t,U):

  # Primero, generar figura con subplot correspondiente
  fig, ax = plt.subplots(figsize = (10,6))
  plt.close()

  ax.set_title(r"heat solution $e^t sin(\pi x)$")

  ax.set_xlim(x.min(), x.max())
  ax.set_ylim(np.floor(U.min()), np.ceil(U.max()))
  ax.set_xlabel(r"$x$")
  ax.set_ylabel(r"$u(x, t)$")

  # Inicializar etiqueta sin texto
  time_label = ax.text(1, 1, "", color = "black", fontsize = 12)
  # Inicializar gráfico sin datos
  line, = ax.plot([], [], color = "black", lw = 2)

  # Definir función de inicialización
  def init():
      line.set_data([], [])
      time_label.set_text("")
      return (line,)

  # Animar función. Esta función se llama secuencialmente con FuncAnimation
  def animate(i):
      line.set_data(x, U[i])
      time_label.set_text(f"t = {t[i]:.2f}")
      return (line,)

  anim = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=len(t), interval=50, blit=True)

  return anim

In [ ]:
# Número de muestras para espacio y tiempo.
# Obs: Podrían tomarse distintos valores de muestreo en tiempo y en espacio, pero se considera un solo valor por simplicidad
dom_samples = 100

# Función para solución analítica
def analytic_diffusion(x,t):
    u = np.exp(-t)*np.sin(np.pi*x)
    return u

# Dominio espacial
x = np.linspace(-1, 1, dom_samples)
# Dominio temporal
t = np.linspace(0, 2, dom_samples)

# Mallado
X, T = np.meshgrid(x, t)
# Evaluar función en mallado
U = analytic_diffusion(X, T)

In [ ]:
# Correr animación y hacer display
anim = animate(x,t,U)
rc("animation", html="jshtml")
anim

## 2. Muestreo del Dominio para Entrenar con PINN
Para entrenar la PINN, vamos a muestrear el dominio usando la estrategia estratificada de Muestreo de Hipercubo Latino (*Latin Hypercube Sampling* - LHS). Este método asegura que se haga un muestreo que cubra uniformemente el espacio de entrada y evita la generación de aglomeraciones mediante la partición del dominio y la obtención aleatorizada de un punto por cada componente de la partición.

Importamos `qmc.LatinHypercube` desde `scipy.stats` y escalamos las muestras para que coincidan con los bordes del dominio. Adicionalmente, se convierten las observaciones y dominio temporal a `torch.tensors` para compatibilizarlos con el modelo de PINN.


In [ ]:
# LHS
sampler = qmc.LatinHypercube(d=2)
sample = sampler.random(n=100)

# Bordes inferior y superior del dominio
l_bounds = [-1, 0]
u_bounds = [ 1, 2]
domain_xt = qmc.scale(sample, l_bounds, u_bounds)

# Tensores de torch
x_ten = torch.tensor(domain_xt[:, 0], requires_grad = True).float().reshape(-1,1)
t_ten = torch.tensor(domain_xt[:, 1], requires_grad = True).float().reshape(-1,1)


fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(domain_xt[:, 0],domain_xt[:, 1], label = 'PDE collocation points')
ax.scatter(domain_xt[:, 0],np.zeros_like(domain_xt[:, 1]), label = 'IC collocation points')
ax.scatter(np.ones_like(domain_xt[:, 0]),domain_xt[:, 1], label = 'BC1 collocation points')
ax.scatter(np.ones_like(domain_xt[:, 1])*-1,domain_xt[:, 1], label = 'BC2 collocation points')
ax.set_title('Collocation points')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$t$')
ax.legend(loc='lower left')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 3. Entrenamiento de la PINN

Entrenamos una red neuronal artificial para aproximar directamente la solución de la EDP, i.e.,

$$
u_{pinn}(t, x; \Theta) \approx u(t,x)
$$

donde $\Theta$ son los parámetros libres (entrenables) de la red. Ahora, usamos `PyTorch` y definimos la red. Para el entrenamiento consideraremos el optimizador ADAM.

In [ ]:
torch.manual_seed(123)

# Hiper-parámetros para el entrenamiento
hidden_layers = [2, 10, 10, 10, 1]
learning_rate = 0.001
training_iter = 20000

In [ ]:
# Error cuadrático medio (Mean Squared Error - MSE)
MSE_func = nn.MSELoss()

# Definir clase de red neuronal con capas y neuronas especificadas por usuario
class NeuralNetwork(nn.Module):

    def __init__(self, hlayers):
        super(NeuralNetwork, self).__init__()

        layers = []
        for i in range(len(hlayers[:-2])):
            layers.append(nn.Linear(hlayers[i], hlayers[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hlayers[-2], hlayers[-1]))

        self.layers = nn.Sequential(*layers)
        self.init_params()

    def init_params(self):
        """Inicialización de parámetros Xavier Glorot
        """
        def init_normal(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight) # Xavier
        self.apply(init_normal)

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Crear instancia de la NN
u_pinn = NeuralNetwork(hidden_layers)
nparams = sum(p.numel() for p in u_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

# Definir optimizador (Adam) para entrenar la red
optimizer = optim.Adam(u_pinn.parameters(), lr=0.001,
                       betas= (0.9,0.999), eps = 1e-8)

### Función de Pérdida Informada con Física

Para entrenar la PINN, definimos las funciones $f_{pde}(t, x)$, $g_{ic}(0)$ and $h_{bc}(0)$ asociadas a la EDP, la condición inicial y condición de borde, respectivamente. Además, reemplazamos la solución $u(t,x)$ por el output de la PINN $u_{pinn}(t,x; \Theta)$:
$$
\begin{align*}
f_{pde}(t,x;u_{pinn}):=& \frac{\partial u_{pinn}}{\partial t} - \frac{\partial^2 u_{pinn}}{\partial x^2} + e^{-t}(\sin(\pi x) - \pi^2 \sin(\pi x)) = 0\\
g_{ic}(0,x;u_{pinn}):=&u_{pinn}(0,x; \Theta) = \sin(\pi x)\\
h_{bc1}(t,-1;u_{pinn}):=&u_{pinn}(t,-1; \Theta) = 0\\
h_{bc2}(t,1;u_{pinn}):=&u_{pinn}(t,1; \Theta) = 0
\end{align*}
$$
Una vez más, consideramos el error cuadrático medio y definimos la función de pérdida con información de la física del modelo:

$$
\begin{align*}
\mathcal{L}(\theta):&= \frac{\lambda_1}{N}\sum_i\left(f_{pde}(t_i, x_i;u_{pinn})-0\right)^2 \quad \text{(Pérdida EDP)}\\
                   & + \frac{\lambda_2}{N} \sum_i(g_{ic}(0,x_i;u_{pinn})-\sin(\pi x_i))^2 \quad \text{(Pérdida CI)}\\
                   & + \frac{\lambda_3}{N} \sum_i(h_{bc1}(t_i,-1;u_{pinn})-0)^2 \quad \text{(Pérdida CB1)}\\
                   & + \frac{\lambda_3}{N} \sum_i(h_{bc2}(t_i,1;u_{pinn})-0)^2 \quad \text{(Pérdida CB2)}\\
\end{align*}
$$

donde $\lambda_{1,2,3}\in\mathbb{R}^+$ son valores de ponderación positivos y $N$ es el número de muestras.

El entrenamiento se realiza minimizando la función de pérdida $\mathcal{L}(\Theta)$:

$$
\min_{\Theta\in\mathbb{R}} \mathcal{L}(\Theta)\rightarrow 0
$$

<div class="alert alert-info"
    style="background-color:#5c5c5c;color:#000000;border-color:#000000">
  <strong>OBSERVACIÓN!</strong> Cuando se incluye la función de datos en la pérdida, estamos empleando un esquema basado en datos (data-driven scheme). En caso contrario, decimos que estamos usando un esquema libre de datos (data-free scheme).
</div>



In [ ]:
# HINT:
def PINN_diffusion_Loss(forward_pass, x_ten, t_ten,
             lambda1 = 1, lambda2 = 1, lambda3 = 1):

    # output de ANN
    domain = torch.cat([t_ten, x_ten], dim = 1)
    u = forward_pass(domain)

    #### EDP ####
    #TODO: Calcular la primera derivada en tiempo
    u_t = ...

    #TODO: Calcular primera y segunda derivada en espacio
    u_x = ...
    u_xx = ...

    #TODO: Calcular pérdida de la EDP

    f_pde = ...
    PDE_loss = ...

    #### CI ####

    #TODO: Calcular pérdida de condición inicial
    g_ic = ...
    IC_loss = ...

    #### CB ####

    #TODO: Calcular pérdida de borde x = -1 (BC1)

    h_bc1 = ...
    BC1_loss = ...

    #TODO: Calcular pérdida de borde x = 1 (BC2)

    bc2 = ...
    h_bc2 = ...
    BC2_loss = ...

    return PDE_loss + IC_loss + BC1_loss + BC2_loss

# Inicializar lista para guardar valores de pérdida
loss_values = []

# Empezar timer
start_time = time.time()

# Entrenar red neuronal
for i in range(training_iter):

    optimizer.zero_grad()   # Reinicializar gradientes para iteración de entrenamiento

    # ingresar x, predecir con PINN y obtener pérdida
    loss = PINN_diffusion_Loss(u_pinn, x_ten, t_ten)

    # Agregar actual valor de pérdida a la lista
    loss_values.append(loss.item())

    if i % 1000 == 0:  # Print cada 1000 iteraciones
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # Paso de retropropagación
    optimizer.step() # Actualizar pesos de la red con optimizador

# Detener timer y obtener tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

In [ ]:
# Graficación
X_ten = torch.tensor(X).float().reshape(-1, 1)
T_ten = torch.tensor(T).float().reshape(-1, 1)
domain_ten = torch.cat([T_ten, X_ten], dim = 1)
U_pred = u_pinn(domain_ten).reshape(dom_samples,dom_samples)

U_true = torch.tensor(U).float()
print(f'Relative error: {relative_l2_error(U_pred, U_true)}')

plot_comparison(U, U_pred, loss_values)

## **Ejercicios**:
1. Agregar caso basado en datos. **Indicación**: Utilice los mismos puntos de colocación de la PINN.
2. Evalúe cómo varía la solución de la PINN aumentando y disminuyendo los pesos `lambdas`.
3. Evalúe cómo varía la solución de la PINN aumentando y disminuyendo la tasa de aprendizaje y el número de iteraciones de entrenamiento.
4. Cambie el número de capas ocultas, neuronas y funciones de activación del modelo de NN.

## **Questions**:
1. ¿Por qué no es necesario incluir datos de la solución en el interior del dominio para resolver el problema de EDP con PINNs?
   <details>
   <summary>Answer</summary>
    En el modelo de difusión, las leyes físicas son bien comprendidas y el problema asociado es bien puesto. El modelo de calor describe cómo una cantidad de interés cambia con el tiempo y cómo esta dinámica se relaciona con el espacio. Las propiedades de los operadores lineales asociados a la EDP de calor hacen que las soluciones al problema de difusión sean únicas y estables, siendo solamente necesarias las condiciones iniciales y de borde para conocerlas completamente al interior del dominio. Este comportamiento hace que no sea necesario tomar un enfoque basado en datos para conocer la solución del problema de difusión, haciendo que la PINN pueda prescindir de información empírica y solo tenga que minimizar residuos asociados a la formulación matemática de la EDP.
   </details>

